<a href="https://colab.research.google.com/github/dipanjanS/mastering-intelligent-agents-langgraph-workshop-dhs2025/blob/main/Module-3-Context-Engineering-for-Agentic-AI-Systems/M3LC1_Building_Agentic_AI_Systems_with_Pre_built_MCP_Servers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building Agentic AI Systems with Pre-built MCP Servers

In this notebook, we explore how to build an **Agentic AI system** by combining a **pre-built MCP server** with LangGraph's AI Agent capabilities.

The system demonstrates how an LLM-based agent can use external tools—served via the [Model Context Protocol (MCP)](https://modelcontextprotocol.io/)—to complete complex tasks.

We use the `markitdown-mcp` server as a concrete example, which exposes a document conversion tool (`convert_to_markdown`) over MCP. The agent uses this tool to extract content from external sources (e.g., web articles) and generate structured markdown reports. The architecture diagram is shown below.

![](https://i.imgur.com/ielptGR.png)

### Key Concepts Covered:
- Setting up and launching a pre-built MCP server (`markitdown-mcp`)
- Using `langchain_mcp_adapters` to connect MCP tools with LangChain agents
- Building a ReAct-style agent using `create_react_agent` from LangGraph
- Running an end-to-end task: convert and summarize content from a URL
- Saving the agent's output as a local markdown file (`document.md`)

This notebook demonstrates how MCP simplifies tool integration for agents, allowing LLMs to reason, act, and interact with external systems through a standard interface, without writing custom wrappers and tools.


## Install Dependencies

In [ ]:
!pip install markitdown-mcp==0.0.1a4 langgraph==0.6.5 langchain==0.3.27 langchain-openai==0.3.29 langchain-community==0.3.27 --quiet

In [ ]:
!pip install langchain-mcp-adapters==0.1.9 --quiet

## Deploy Pre-built Markitdown MCP Server

In [ ]:
!nohup markitdown-mcp --http --host 127.0.0.1 --port 8011 > server_output.log 2>&1 &

![](https://i.imgur.com/cHB4UzQ.png)

## Get OpenAI Key Environment File

In [ ]:
!gdown 1dyWtBg0RCg_WqzsuMWhvJwrN8_GGDmbI

## Add your OpenAI Key in the Environment File

Open up the `.env` file and enter your OpenAI key in the place shown below and then save the file

![](https://i.imgur.com/ThkufIW.png)

## Create your Client Content Analyst & Report Generator AI Agent

In [ ]:
%%writefile client_agent.py

from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
import asyncio
from dotenv import load_dotenv
from langchain_core.messages import SystemMessage, HumanMessage

load_dotenv()
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

async def main():
    client = MultiServerMCPClient({
        "markitdown": {
            "url": "http://localhost:8011/mcp",
            "transport": "streamable_http"
        }

    })

    tools = await client.get_tools()
    print("Discovered tools:", [tool.name for tool in tools])

    AGENT_PROMPT_TXT = """Act as a helpful assistant.
    Use the tools available to extract information as necessary.
    Summarize the key aspects and provide an executive report.
    """

    SYS_PROMPT = SystemMessage(content=AGENT_PROMPT_TXT)
    agent = create_react_agent(model=llm, tools=tools, prompt=SYS_PROMPT)

    # Sample queries
    print('\nUsing capabilities for document extraction:')
    source = "https://www.ibm.com/think/topics/agentic-ai"
    prompt = """Create a report on the following article
                Source: {src}"""
    prompt = prompt.format(src=source)
    result = await agent.ainvoke({"messages": [{"role": "user", "content": prompt}]})
    with open("document.md", 'w') as f:
        f.write(result["messages"][-1].content)
    print('Report has been created')

if __name__ == "__main__":
    asyncio.run(main())

## Run the Agent

We can run the agent as shown in the following snapshot in the terminal

![](https://i.imgur.com/GuDpKOU.png)

Then you can open up the `document.md` report file created by the agent as shown here.

![](https://i.imgur.com/gYKtyaL.png)